In [3]:
from google.colab import files

In [4]:
uploaded = files.upload()

Saving hotel_reviews.csv to hotel_reviews.csv


In [5]:
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

User uploaded file "hotel_reviews.csv" with length 14986511 bytes


In [6]:
uploaded

In [7]:
import pandas as pd
import re
import string
import warnings
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
from imblearn.over_sampling import SMOTE
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from textblob import TextBlob
from wordcloud import WordCloud
import nltk

from sklearn.model_selection import train_test_split, KFold, cross_val_score

from sklearn.linear_model import LogisticRegression 
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, plot_confusion_matrix, balanced_accuracy_score

from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.base import BaseEstimator, TransformerMixin

from textblob import TextBlob
from wordcloud import WordCloud

from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import RandomUnderSampler, TomekLinks
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.pipeline import Pipeline, make_pipeline

from sklearn_pandas import DataFrameMapper
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Binarizer, FunctionTransformer 

#from pickle import dumps
from pickle import load

from dill import dumps as dp
from dill import loads as ld 
from cloudpickle import dump
import joblib

import warnings
warnings.filterwarnings('ignore')


In [8]:
customer_review = pd.read_csv('hotel_reviews.csv')
customer_review.head()

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5


In [ ]:
customer_review.info()

In [ ]:
customer_review.shape

In [ ]:
customer_review.describe()

In [ ]:
customer_review.isnull().sum()

In [ ]:
customer_review.duplicated()

In [ ]:
len(customer_review[customer_review.duplicated()])

In [ ]:
customer_review["Rating"].value_counts()

In [ ]:
sns.countplot(x = customer_review['Rating'],edgecolor='black',linewidth=1, palette=['#432371',"#FAAE7B","#940036","#00948b","#003f94"]);
plt.title('Count of Ratings')
plt.show()

In [ ]:
customer_review['Rating'].value_counts().plot.pie(explode =[0.05,0.04,0.03,0.02,0.01],autopct ='%1.1f%%',startangle = 90,figsize=(10,10))
plt.title('Customer Rating')

In [ ]:
sns.distplot(customer_review['Rating'])

In [ ]:
#Checking the most used words from reviews & plotting it using WordCloud
def wordCloud_generator(data, title=None):
    wordcloud = WordCloud(width = 1000, height = 750, max_words=300,
                          background_color ='white',
                          min_font_size = 10
                         ).generate(" ".join(data.values))
                        
    plt.figure(figsize = (6, 5), facecolor = None) 
    plt.imshow(wordcloud, interpolation='bilinear') 
    plt.axis("off") 
    plt.tight_layout(pad = 0) 
    plt.title(title,fontsize=25)
    plt.show() 

In [ ]:
#Plotting the wordcloud on original data for all ratings
wordCloud_generator(data=customer_review['Review'], title="Most used words in reviews\n")

In [ ]:
#Checking the most used words from reviews & plotting it using WordCloud
def wordCloud_generator(data, title=None):
    wordcloud = WordCloud(width = 1000, height = 750, max_words=300,
                          background_color ='white',
                          min_font_size = 10
                         ).generate(" ".join(data.values))
                        
    plt.figure(figsize = (6, 5), facecolor = None) 
    plt.imshow(wordcloud, interpolation='bilinear') 
    plt.axis("off") 
    plt.tight_layout(pad = 0) 
    plt.title(title,fontsize=25)
    plt.show() 

In [ ]:
#Plotting the wordcloud on original data for all highest rating (5)
wordCloud_generator(data=customer_review[customer_review['Rating']==5]['Review'], title="Most used words for highest rating\n")

In [ ]:
#Checking the most used words from reviews & plotting it using WordCloud
def wordCloud_generator(data, title=None):
    wordcloud = WordCloud(width = 1000, height = 750, max_words=300,
                          background_color ='white',
                          min_font_size = 10
                         ).generate(" ".join(data.values))
                        
    plt.figure(figsize = (6, 5), facecolor = None) 
    plt.imshow(wordcloud, interpolation='bilinear') 
    plt.axis("off") 
    plt.tight_layout(pad = 0) 
    plt.title(title,fontsize=25)
    plt.show() 

In [ ]:
#Plotting the wordcloud on original data for all lowest rating (1)
wordCloud_generator(data=customer_review[customer_review['Rating']==1]['Review'],title="Most used words for lowest rating\n")

In [ ]:
#Checking the length of words in a review for all the rating
plt.figure(figsize=(8,5))
sns.scatterplot(x=customer_review['Review'].apply(len), y=customer_review['Rating'], data=customer_review, palette='crest', hue='Rating')
plt.title('Length of Review vs Rating')
plt.yticks([1,2,3,4,5])
plt.show()

In [ ]:
customer_review1 = customer_review.copy()
customer_review1

In [ ]:
pip install nltk


In [ ]:
nltk.download('stopwords')

In [ ]:
nltk.download('punkt')

In [ ]:
nltk.download('wordnet')

In [ ]:
nltk.download('omw-1.4')

In [ ]:
#Object Instantiation
stop_words = nltk.corpus.stopwords.words('english')
stpw = ENGLISH_STOP_WORDS
wnl = WordNetLemmatizer()

In [ ]:
# defined a function for cleaning all reviews in the data set

def clean_text(text):
    
    text = re.sub(r'\w*\d\w*', '', str(text)).strip() #Removing numeric attached with words and return only alpha 
    
    text = re.sub("[\d]+", "", str(text))  # Removing the strings which contains unnecessary digits in data
    
    text = re.sub(r"won\'t", "will not", str(text))      # Replace contraction words
    text = re.sub(r"can\'t", "can not", str(text))      # Replace contraction words
    text = re.sub(r"ca n\'t", "can not", str(text))      # Replace contraction words
    text = re.sub(r"wo n\'t", "will not", str(text))      # Replace contraction words
    text = re.sub(r"\'t've", " not have", str(text))     # Replace contraction words
    text = re.sub(r"\'d've", " would have", str(text))   # Replace contraction words
    text = re.sub(r"\'cause", " because", str(text))     # Replace contraction words
    
    text = re.sub(r"n\'t", " not", str(text))     # Replace contraction words
    text = re.sub(r"\'re", " are", str(text))     # Replace contraction words
    text = re.sub(r"\'s", " is", str(text))       # Replace contraction words
    text = re.sub(r"\'d", " would", str(text))    # Replace contraction words
    text = re.sub(r"\'ll", " will", str(text))    # Replace contraction words
    text = re.sub(r"\'t", " not", str(text))      # Replace contraction words
    text = re.sub(r"\'ve", " have", str(text))    # Replace contraction words
    text = re.sub(r"\'m", " am", str(text))       # Replace contraction words

    text = re.sub(r"n\'t", " not", str(text))     # Replace contraction words
    text = re.sub(r"\'re", " are", str(text))     # Replace contraction words
    text = re.sub(r"\'s", " is", str(text))       # Replace contraction words
    text = re.sub(r"\'d", " would", str(text))    # Replace contraction words
    text = re.sub(r"\'ll", " will", str(text))    # Replace contraction words
    text = re.sub(r"\'t", " not", str(text))      # Replace contraction words
    text = re.sub(r"\'ve", " have", str(text))    # Replace contraction words
    text = re.sub(r"\'m", " am", str(text))       # Replace contraction words
    
    text = re.sub(r'[^\w\s]', " ", str(text))    # Removing white spaces and returning only alpha characters
    
    text = text.translate(str.maketrans('','',string.punctuation)) # Remove Punctuations
    
    text = re.sub('[%s]' % re.escape(string.punctuation), '', str(text)) # Remove Punctuations
    
    text = ' '.join( [w for w in text.split() if len(w)>1] ) # Removing an unnecessary single character from sentences
    
    text = text.split() #Splitting each word
    
    text = " ".join([word for word in text if word.lower().strip() not in stop_words]) #Removing stopwords, white spaces, making words lower and joining into text
    
    text = text.split() # Splitting each word
    
    text = " ".join([word for word in text if word not in stpw]) # Removing extra stopwords if any
    
    text = nltk.word_tokenize(text)  # Tokenising each words 
       
    text = " ".join([wnl.lemmatize(w, 'v') for w in text])  # Applying Lemmatization

    text = ' '.join(dict.fromkeys(text.split())) #Remove duplicate words from a sentence
    
    return text

In [ ]:
#Testing purpose - reviewing the original reviews before text preprocessing
customer_review1['Review'][2]

In [ ]:
len(customer_review['Review'][2])

In [ ]:
#Testing purpose - reviewing the original reviews after text preprocessing
clean_text(customer_review1['Review'][2])

In [ ]:
#Testing purpose - reviewing the original reviews after text preprocessing
len(clean_text(customer_review1['Review'][2]))

In [ ]:
customer_review1['Fully_Clean_Text'] = customer_review1['Review'].apply(lambda x: clean_text(x))
customer_review1

In [ ]:
#Testing purpose - Verifying the sample review
len(customer_review1['Fully_Clean_Text'][2])

In [ ]:
customer_review1['Length'] = customer_review1['Review'].apply(len)
new_length = customer_review1['Length'].sum()

In [ ]:
customer_review1['Lengths'] = customer_review1['Fully_Clean_Text'].apply(len)
new_lengths = customer_review1['Lengths'].sum()

In [ ]:
#Visualizing the length of reviews
sns.barplot(data=customer_review1, x='Rating', y='Length')
plt.title('Length of Reviews w.r.t Ratings\n')
plt.show()

In [ ]:
#Comparing the length of reviews before & after text preprocessing
print('\033[1m'+ "Total Length of Review Comparison" + '\033[0m')
print('=======================================')
print("Before Text Preprocessing : {}".format(new_length))
print('---------------------------------------')
print("After Text Preprocessing  :{}".format(new_lengths))
print('=======================================')

Note:
We have successfully completed the text preprocessing.
Let's analyse our data further w.r.t Sentiment Analysis.
Sentiment Analysis
Rule Based

VADER (Valence Aware Dictionary for Sentiment Reasoning)
TextBlob
Lexicons with polarity scores - Affin
Model Based

Logistic Regression
Naive Bayes
Distilbert - Pretrained hugging face model
Rule Based Methods
Using NLTK's VADER - Sentiment Intensity Analyzer
Vader is a lexicon and rule based sentiment analysis tool specifically calibrated to sentiments most commonly expressed on social media platforms. When calculating a polarity score Vader outputs four metrics: compound, negative, neutral and positive. The compound score calculates the sum of all lexicon ratings which is normalized between -1 (most negative) and +1 (most positive).

In [ ]:
##Object Instantiation
nltk.download('vader_lexicon')
sia = SentimentIntensityAnalyzer()

In [ ]:
#Defining a function for Sentiment Analysis
def fetch_sentiment_using_SIA(text):
    sia = SentimentIntensityAnalyzer()
    polarity_scores = sia.polarity_scores(text)
    return 'neg' if polarity_scores['neg'] > polarity_scores['pos'] else 'pos'

In [ ]:
#Applying the VADER function to entire data
customer_review1['SIA_Vader'] = customer_review1.Fully_Clean_Text.apply(lambda x: fetch_sentiment_using_SIA(x))

In [ ]:
#Reviewing the data
customer_review1[['Fully_Clean_Text', 'Rating', 'SIA_Vader']]

In [ ]:
#Drawn a positive sample for testing the sentiment
customer_review1['Fully_Clean_Text'][20487]

In [ ]:
sia.polarity_scores(customer_review1['Fully_Clean_Text'][20487])

In [ ]:
#Drawn a negative sample for testing the sentiment
customer_review1['Fully_Clean_Text'][20489]

In [ ]:
#Testing the sample
sia.polarity_scores(customer_review1['Fully_Clean_Text'][20489])

In [ ]:
%matplotlib inline

In [ ]:
#Visualizing the Vader Sentiment Analysis
plt.figure(figsize=(8,5))

plt.title("Visualizing the VADER Sentiment w.r.t Ratings")

ax = sns.countplot(data = customer_review1 , x = 'Rating', hue = 'SIA_Vader')

for i in ax.containers:
    ax.bar_label(i,)
plt.show()
    
        
print('Total Count of Sentiments\n')

print(customer_review1['SIA_Vader'].value_counts(['SIA_Vader'])*100, '\n')

In [ ]:
#Applying the TextBlob to entire data
customer_review1['TxtB_Polarity'] = customer_review1['Fully_Clean_Text'].apply(lambda x: TextBlob(x).sentiment[0])

In [ ]:
#Reviewing the data
customer_review1[['Fully_Clean_Text', 'Rating', 'TxtB_Polarity']]

In [ ]:
#Drawn a positive sample for testing the sentiment
txt = TextBlob(customer_review1['Fully_Clean_Text'][20487])

In [ ]:
#Testing the sample
txt.polarity

In [ ]:
#Testing the sample
txt.sentiment_assessments

In [ ]:
#Drawn a negative sample for testing the sentiment
txt2 = TextBlob(customer_review1['Fully_Clean_Text'][20489])

In [ ]:
#Testing the sample
txt2.polarity

In [ ]:
#Testing the sample
txt2.sentiment_assessments

In [ ]:
#Visualizing the TextBlob Sentiment Analysis
plt.figure(figsize=(8,5))

ax = sns.barplot(data= customer_review1, x = 'Rating', y = 'TxtB_Polarity')

plt.title("Visualizing the TextBlob Sentiment w.r.t Ratings")

plt.show()

In [ ]:
!pip install afinn

In [ ]:
# Emotion Lexicon - Affin
from afinn import Afinn

In [ ]:
#Object Instantiation
afn = Afinn()

In [ ]:
#Drawn a negative sample for testing the sentiment
nn = customer_review1['Fully_Clean_Text'][20490]

In [ ]:
#Testing the negative sample
afn.score(nn)

In [ ]:
#Drawn a positive sample for testing the sentiment
pp = customer_review1['Fully_Clean_Text'][20487]

In [ ]:
#Testing the positive sample
afn.score(pp)

In [ ]:
#Applying afinn to dataset
customer_review1['Afinn_score'] = customer_review1['Fully_Clean_Text'].apply(lambda x: afn.score(x))

In [ ]:
#Reviewing the data
customer_review1[['Fully_Clean_Text',  'Rating', 'Afinn_score']]

In [ ]:
#Visualizing the Afinn Sentiment Analysis

plt.figure(figsize=(15,8))

ax = sns.violinplot(data= customer_review1, x = 'Rating', y = 'Afinn_score')

plt.title("Visualizing the TextBlob Sentiment w.r.t Ratings")

plt.show()

In [ ]:
#Defining a function for Sentiment Analysis
def condition(x):
    if x>=20:
        return "Pos"
    else:
        return 'Neg'

In [ ]:
#Applying condition on Afinn score 
customer_review1['Affin_Sent'] = customer_review1['Afinn_score'].apply(condition)

In [ ]:
#Reviewing the data
customer_review1[['Fully_Clean_Text',  'Rating', 'Afinn_score', 'Affin_Sent']]

In [ ]:
#Count of Afinn sentiment
customer_review1['Affin_Sent'].value_counts()

In [ ]:
#Analysing the data for rating = 1 with lowest score
customer_review1[customer_review1['Rating']==1].min()['Afinn_score']

In [ ]:
#Analysing the data for rating = 1 with highest score
customer_review1[customer_review1['Rating']==1].max()['Afinn_score']

In [ ]:
#Analysing the data for rating = 5 with lowest score
customer_review1[customer_review1['Rating']==5].min()['Afinn_score']

In [ ]:
#Analysing the data for rating = 5 with lowest score
customer_review1[customer_review1['Rating']==5].max()['Afinn_score']

In [ ]:
#Converting the ratings into two labels of Positive & Negative

customer_review1['Target'] = customer_review1['Rating'].apply(lambda x: 1 if x > 3 else 0)

In [ ]:
#Reference for original data for encoded value
customer_review1.head()

In [ ]:
#Re-checking the dtype for all columns
customer_review1.info()

In [ ]:
#Checking the ratings
plt.figure(figsize=(12,6))

g = plt.pie(round(customer_review1.Target.value_counts(normalize=True)*100,2),explode=(0.025,0.025), 
            labels=['Positive - 1', 'Negative - 0'], colors=["c","m"],
            autopct="%1.1f%%", startangle=180)

plt.title("% Distributions by Review Type")

plt.show()

In [ ]:
#TF-IDF features
vectorizer = TfidfVectorizer()

In [ ]:
#Splitting the variables into features & target
X = customer_review1['Fully_Clean_Text']
y = customer_review1['Target']

In [ ]:
#Checking the count of target variable
customer_review1['Target'].value_counts()

In [ ]:
#Segregating data into train & test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.3, shuffle=True, stratify=y)

In [ ]:
#Applying the TF-IDF vectorizer to train data
X_train = vectorizer.fit_transform(X_train)

In [ ]:
#Applying the TF-IDF vectorizer to test data
X_test = vectorizer.transform(X_test)

In [ ]:
#Building the different classification models with default parameters
lr_mod  = LogisticRegression(random_state=0, max_iter=1000, class_weight='balanced')
nb_mod  = MultinomialNB()

In [ ]:
%%time
lr_mod.fit(X_train, y_train)
nb_mod.fit(X_train, y_train)

In [ ]:
%%time
y_pred_lr  = lr_mod.predict(X_test)
y_pred_nb = nb_mod.predict(X_test)

In [ ]:
#Checking the count of test variable - target
y_test.value_counts()

In [ ]:
#Checking the accuracy for all models
cc  = [y_pred_lr, y_pred_nb]
mod = ['Logistic Regression', 'Naive Bayes']

print('Accuracy Scores for all models','\n')

for i,j in zip(cc, mod):
    print('===========================')
    print(j, ':', balanced_accuracy_score(y_test, i).round(2))
    print('===========================', '\n')

In [ ]:
#Plotting Confusion Matrix for all models
dd  = [lr_mod, nb_mod]
mod = ['Logistic Regression', 'Naive Bayes']

print('Confusion Matrix for all models')

for i,j in zip(dd, mod):
    print('\n','========================================', '\n')
    print(j)
    plot_confusion_matrix(i, X_test, y_test, display_labels= ['Neg', 'Pos'])   
    plt.show()

In [ ]:
#Classification Report for all models
cc  = [y_pred_lr, y_pred_nb]
mod = ['Logistic Regression', 'Naive Bayes']

print('Classification Report for all models','\n')

for i,j in zip(cc, mod):
    print('========================================================')
    print(j,'\n\n' , classification_report(y_test, i))
    print('========================================================', '\n')

In [ ]:
#Transforming the data to X & y
X = vectorizer.fit_transform(X)
y = y

In [ ]:
#Object Instantiation
sm = SMOTE(random_state=0)

In [ ]:
#Resampling the dataset with smote
X_res, y_res = sm.fit_resample(X, y)

In [ ]:
#Rechecking the count of target variable after smote
print('X_res after resampling shape : ' , X_res.shape)
print('y_res after resampling shape : ' , y_res.shape, '\n')
print(y_res.value_counts())

Building models with balanced data (smote_resampled data)

In [ ]:
#Splitting data into training and testing dataset with 80:20 ratio
X_train_res, X_test_res, y_train_res, y_test_res = train_test_split(X_res, y_res, random_state=0, test_size=0.35, stratify=y_res)

In [ ]:
#Checking the size of features
X_train_res.shape, X_test_res.shape

Model Building with default parameters & smote_resampled data

In [ ]:
lr_mod2  =LogisticRegression(random_state=0, max_iter=1000)
nb_mod2  = MultinomialNB()

Model Training

In [ ]:
%%time
lr_mod2.fit(X_train_res, y_train_res)
nb_mod2.fit(X_train_res, y_train_res)

Model Testing & Evaluation

In [ ]:
%%time
y_pred_lr2  = lr_mod2.predict(X_test_res)
y_pred_nb2  = nb_mod2.predict(X_test_res)

In [ ]:
#Checking the accuracy for all models
cc  = [y_pred_lr2, y_pred_nb2]
mod = ['Logistic Regression', 'Naive Bayes']

print('Accuracy Scores for all models','\n')

for i,j in zip(cc, mod):
    print('===========================')
    print(j, ':', balanced_accuracy_score(y_test_res, i).round(2))
    print('===========================', '\n')

In [ ]:
#Plotting Confusion Matrix for all models
mm = ['Logistic Regression', 'Naive Bayes']
dd = [lr_mod2 ,nb_mod2]

print('Confusion Matrix for all models')

for i,j in zip(dd, mm):
    print('\n','========================================', '\n')
    print(j)
    plot_confusion_matrix(i, X_test_res, y_test_res, display_labels= ['Neg', 'Pos'])   
    plt.show()

In [ ]:
#Classification Report for all models
cc = [y_pred_lr2, y_pred_nb2]
mm = ['Logistic Regression', 'Naive Bayes']

for i,j in zip(cc, mm):
    print('======================================================')
    print(j, '\n', classification_report(y_test_res, i))
    print('======================================================', '\n')

In [ ]:
#Comparing the models imbalance & model smote_balance data with balanced accuracy score
mod_comp = pd.DataFrame(index=mm, columns=['Model1', 'Model2'], data={'Model1': [balanced_accuracy_score(y_test ,y_pred_lr).round(2), balanced_accuracy_score(y_test ,y_pred_nb).round(2)], 
                                                                      'Model2': [balanced_accuracy_score(y_test_res ,y_pred_lr2).round(2), balanced_accuracy_score(y_test_res ,y_pred_nb2).round(2)]
                                                                       })
mod_comp['%Change in Accuracy'] = (mod_comp['Model2'] - mod_comp['Model1'])*100
mod_comp.sort_values(by='Model2', ascending=False)

From above analysis,


We can say that after applying the sampling method - smote technique to our data, the accuracy has improved by more than 30% for Naive Bayes model.
Also, the recall class 0, 1 has been been improved.
But, smote has simply oversampled minority class ("Negatives - 0" in our case).

**Adaptive Synthetic Sampling (ADASYN)**

It involves generating synthetic samples inversely proportional to the density of the examples in the minority class. That is, generate more synthetic examples in regions of the feature space where the density of minority examples is low, and fewer or none where the density is high.

In [ ]:
#Object Instantiation
ad = ADASYN(random_state=0)

In [ ]:
#Resampling the dataset with adasyn
X_ad, y_ad = ad.fit_resample(X, y)

In [ ]:
#Rechecking the count of target variable after adasyn
print('X_res after resampling shape : ' , X_ad.shape)
print('y_res after resampling shape : ' , y_ad.shape, '\n')
print(y_ad.value_counts())

Building models with balanced data (ADASYN)

In [ ]:
#Splitting data into training and testing dataset with 80:20 ratio
X_train_ad, X_test_ad, y_train_ad, y_test_ad = train_test_split(X_ad, y_ad, random_state=0, test_size=0.35, stratify=y_ad)

In [ ]:
#Checking the size of features
X_train_ad.shape, X_test_ad.shape

Model Building with default parameters & adasyn_resampled data

In [ ]:
lr_mod3  = LogisticRegression(random_state=0, max_iter=1000)
nb_mod3  = MultinomialNB()

Model Training

In [ ]:
%%time
lr_mod3.fit(X_train_ad, y_train_ad)
nb_mod3.fit(X_train_ad, y_train_ad)

Model Testing & Evaluation

In [ ]:
%%time
y_pred_lr3  = lr_mod3.predict(X_test_ad)
y_pred_nb3  = nb_mod3.predict(X_test_ad)

In [ ]:
#Testing the roc-auc score for all models
cc = [y_pred_lr3, y_pred_nb3]
mm = ['Logistic Regression', 'Naive Bayes']

print('ROC-AUC score for all models')

for i,j in zip(cc, mm):
    print('\n==============================')
    print(j, ':', balanced_accuracy_score(y_test_ad, i).round(2))
    print('==============================', '\n')

In [ ]:
#Plotting Confusion Matrix for all models
mm = ['Logistic Regression', 'Naive Bayes']
dd = [lr_mod3, nb_mod3]

print('Confusion Matrix for all models')

for i,j in zip(dd, mm):
    print('\n','========================================', '\n')
    print(j)
    plot_confusion_matrix(i, X_test_ad, y_test_ad, display_labels= ['Negative', 'Positive'])   
    plt.show()

In [ ]:
#Classification Report for all models
cc = [y_pred_lr3, y_pred_nb3]
mm = ['Logistic Regression', 'Naive Bayes']

for i,j in zip(cc, mm):
    print('======================================================')
    print(j, '\n', classification_report(y_test_ad, i))
    print('======================================================', '\n')

In [ ]:
#Comparing the models imbalance & model adasyn_balance data with balanced accuracy score
mod_comp = pd.DataFrame(index=mm, columns=['Model1', 'Model2'], data={'Model1': [balanced_accuracy_score(y_test ,y_pred_lr).round(2), balanced_accuracy_score(y_test ,y_pred_nb).round(2)], 
                                                                      'Model2': [balanced_accuracy_score(y_test_ad ,y_pred_lr3).round(2), balanced_accuracy_score(y_test_ad ,y_pred_nb3).round(2)]
                                                                       })
mod_comp['%Change in Accuracy'] = mod_comp['Model2'] - mod_comp['Model1']
mod_comp.sort_values(by='Model2', ascending=False)

**From above analysis,**


There's a minimal improvement in minority class compared to smote technique, even though the accuracy remained the same more or less.
Let's try with the hybrid sampling technique

**Random Under Sampler**

A simple under-sampling technique is to under-sample the majority class randomly and uniformly. This can potentially lead to the loss of information. But if the examples of the majority class are near to others, this method might yield good results.

In [ ]:
#Object Instantiation
rs = RandomUnderSampler(random_state=0)

In [ ]:
#Resampling the dataset with random-under-sampler
X_rs, y_rs = rs.fit_resample(X, y)

In [ ]:
#Rechecking the count of target variable after random-under-sampler
print('X_res after resampling shape : ' , X_rs.shape)
print('y_res after resampling shape : ' , y_rs.shape, '\n')
print(y_rs.value_counts())

In [ ]:
#Splitting data into training and testing dataset with 80:20 ratio
X_train_rs, X_test_rs, y_train_rs, y_test_rs = train_test_split(X_rs, y_rs, random_state=0, test_size=0.35, stratify=y_rs)

In [ ]:
#Checking the size of features
X_train_rs.shape, X_test_rs.shape

In [ ]:
lr_mod4  = LogisticRegression(random_state=0, max_iter=1000)
nb_mod4  = MultinomialNB()

**Model Training**

In [ ]:
%%time
lr_mod4.fit(X_train_rs, y_train_rs)
nb_mod4.fit(X_train_rs, y_train_rs)

**Model Testing & Evaluation**

In [ ]:
%%time
y_pred_lr4  = lr_mod4.predict(X_test_rs)
y_pred_nb4  = nb_mod4.predict(X_test_rs)

In [ ]:
#Testing the balanced accuracy for all models
cc = [y_pred_lr4, y_pred_nb4]
mm = ['Logistic Regression', 'Naive Bayes']

print('Balanced Accuracy score for all models')

for i,j in zip(cc, mm):
    print('\n==============================')
    print(j, ':', balanced_accuracy_score(y_test_rs, i).round(2))
    print('==============================', '\n')

In [ ]:
#Plotting Confusion Matrix for all models
mm = ['Logistic Regression', 'Naive Bayes']
dd = [lr_mod4, nb_mod4]

print('Confusion Matrix for all models')

for i,j in zip(dd, mm):
    print('\n','========================================', '\n')
    print(j)
    plot_confusion_matrix(i, X_test_rs, y_test_rs, display_labels= ['Negative', 'Positive'])   
    plt.show()

In [ ]:
#Classification Report for all models
cc = [y_pred_lr4, y_pred_nb4]
mm = ['Logistic Regression', 'Naive Bayes']

for i,j in zip(cc, mm):
    print('======================================================')
    print(j, '\n', classification_report(y_test_rs, i))
    print('======================================================', '\n')

In [ ]:
#Comparing the models imbalance & model random-under-sampler_balance data with balanced accuracy score
mod_comp = pd.DataFrame(index=mm, columns=['Model1', 'Model2'], data={'Model1': [balanced_accuracy_score(y_test ,y_pred_lr).round(2), balanced_accuracy_score(y_test ,y_pred_nb).round(2)], 
                                                                      'Model2': [balanced_accuracy_score(y_test_rs ,y_pred_lr4).round(2), balanced_accuracy_score(y_test_rs ,y_pred_nb4).round(2)]
                                                                       })
mod_comp['%Change in Accuracy'] = (mod_comp['Model2'] - mod_comp['Model1'])*100
mod_comp.sort_values(by='Model2', ascending=False)

**From above analysis,**

The majority class data has been brought down 
close to minority class.

There's a decrement in the accuracy for Logistic Regression as well as the Naive bayes accuracy scores compared to smote resampled data accuracy score.

Let's try with the another down sampling technique.

**Tomek Links**

Tomek Links ability to remove the data that are identified as Tomek links from the majority class (that is, samples of data from the majority class that is closest with the minority class data)

In [ ]:
#Object Instantiation
tl = TomekLinks()

In [ ]:
#Resampling the dataset with smote-tomek
X_tl, y_tl = tl.fit_resample(X, y)

In [ ]:
##Rechecking the count of target variable after tomek
print('X_res after resampling shape : ' , X_tl.shape)
print('y_res after resampling shape : ' , y_tl.shape, '\n')
print(y_tl.value_counts())

**Building models with balanced data (SMOTE + TOMEK)**

In [ ]:
#Splitting data into training and testing dataset with 80:20 ratio
X_train_tl, X_test_tl, y_train_tl, y_test_tl = train_test_split(X_tl, y_tl, random_state=0, test_size=0.35, stratify=y_tl)

In [ ]:
#Checking the size of features
X_train_tl.shape, X_test_tl.shape

**Model Building with default parameters & smote-tomek_resampled data**

In [ ]:
lr_mod5  = LogisticRegression(random_state=0, max_iter=1000)
nb_mod5  = MultinomialNB()

**Model Training**

In [ ]:
%%time
lr_mod5.fit(X_train_tl, y_train_tl)
nb_mod5.fit(X_train_tl, y_train_tl)

**Model Testing & Evaluation**

In [ ]:
%%time
y_pred_lr5  = lr_mod5.predict(X_test_tl)
y_pred_nb5  = nb_mod5.predict(X_test_tl)

In [ ]:
#Testing the accuracy score for all models
cc = [y_pred_lr5, y_pred_nb5]
mm = ['Logistic Regression', 'Naive Bayes']

print('Balanced Accuracy score for all models')

for i,j in zip(cc, mm):
    print('\n==============================')
    print(j, ':', balanced_accuracy_score(y_test_tl, i).round(2))
    print('==============================', '\n')

In [ ]:
#Plotting Confusion Matrix for all models
mm = ['Logistic Regression', 'Naive Bayes']
dd = [lr_mod5, nb_mod5]

print('Confusion Matrix for all models')

for i,j in zip(dd, mm):
    print('\n','========================================', '\n')
    print(j)
    plot_confusion_matrix(i, X_test_tl, y_test_tl, display_labels= ['Negative', 'Positive'])   
    plt.show()

In [ ]:
#Classification Report for all models
cc = [y_pred_lr5, y_pred_nb5]
mm = ['Logistic Regression', 'Naive Bayes']

for i,j in zip(cc, mm):
    print('======================================================')
    print(j, '\n', classification_report(y_test_tl, i))
    print('======================================================', '\n')

In [ ]:
#Comparing the models imbalance & model smote_balance data with balanced accuracy score
mod_comp = pd.DataFrame(index=mm, columns=['Model1', 'Model2'], data={'Model1': [balanced_accuracy_score(y_test ,y_pred_lr).round(2), balanced_accuracy_score(y_test ,y_pred_nb).round(2)], 
                                                                      'Model2': [balanced_accuracy_score(y_test_tl ,y_pred_lr5).round(2), balanced_accuracy_score(y_test_tl ,y_pred_nb5).round(2)]
                                                                       })
mod_comp['%Change in Accuracy'] = (mod_comp['Model2'] - mod_comp['Model1'])*100
mod_comp.sort_values(by='Model2', ascending=False)

**From above analysis,**

We can see there's a decrement in minority recall class 0, for Logistic Regression & Naive Bayes model compared to other sampling techniques.

Both, these models have under performed with this technique.

**Note:**

After analysis of sampling techniques, we will choose smote resampled data for further analysis for machine learning based sentiment analysis.

Hence, as we observed the accuracy, precision and recall score have been improved using smote.

## **ML Model Stability & Performance**

**Cross Validation on original data**

In [ ]:
kf = KFold(n_splits=15, shuffle=True, random_state=0)

In [ ]:
cross_val_score(lr_mod, X, y, cv=kf, scoring='accuracy').mean().round(3)*100

In [ ]:
cross_val_score(lr_mod, X, y, cv=kf, scoring='accuracy').std().round(3)*100

In [ ]:
cross_val_score(nb_mod, X, y, cv=kf, scoring='accuracy').mean().round(3)*100

In [ ]:
cross_val_score(nb_mod, X, y, cv=kf, scoring='accuracy').std().round(3)*100

**The cross validation score is around 89% for Logistic Regression with 1% deviation, 77% for Naive Bayes model with 1.2% deviation on original dataset respectively.**

**Cross Validation on smote data**

In [ ]:
cross_val_score(lr_mod2, X_res, y_res, cv=kf, scoring='accuracy').mean().round(3)*100

In [ ]:
cross_val_score(lr_mod2, X_res, y_res, cv=kf, scoring='accuracy').std().round(3)*100

In [ ]:
cross_val_score(nb_mod2, X_res, y_res, cv=kf, scoring='accuracy').mean().round(3)*100

In [ ]:
cross_val_score(nb_mod2, X_res, y_res, cv=kf, scoring='accuracy').std().round(3)*100

**The cross validation score is around 92% for Logistic Regression with 0.7 % deviation, 88% for Naive Bayes model with around 0.6 % deviation on smote resampled dataset respectively.**

Let's, try building model using deep learning!!
# **Model Based - Deep Learning pretrained models**


Distilbert - Sentiment Analysis
Model Description: This model is a fine-tune checkpoint of DistilBERT-base-uncased, fine-tuned on SST-2. This model reaches an accuracy of 91.3 on the dev set (for comparison, Bert bert-base-uncased version reaches an accuracy of 92.7) 

In [ ]:
!pip install transformers

In [ ]:
 #Import required libraries
from transformers import AutoTokenizer, pipeline, TFDistilBertForSequenceClassification
from transformers import AutoModelForSequenceClassification, TFAutoModelForSequenceClassification 
from transformers import DistilBertModel, DistilBertConfig, TFTrainer
from transformers import DistilBertTokenizer, TFPreTrainedModel
from scipy.special import softmax
import torch

In [ ]:
pip install tokenizer

In [ ]:
#Model Building 
model_name = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained (model_name)
# tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
tf_model = TFAutoModelForSequenceClassification.from_pretrained(model_name, from_pt=True)
classifier1 = pipeline('sentiment-analysis', model=tf_model, tokenizer=tokenizer)

In [ ]:
classifier1

In [ ]:
#Testing the model - sarcastic sentence
classifier1('The bed was in such a good condition, I slept on the floor.')

In [ ]:
#Testing the model - sarcastic sentence
classifier1('the food will bring nausea and motion sickness; it was that deliciously sick.')

In [ ]:
#Testing the model - sarcastic sentence
classifier1('It was a fucking amazing experience!! Will definitely visit this hotel again.')

In [ ]:
#Testing the model - negative sentence
classifier1('I am not feeling well.')

In [ ]:
#Saving the model
classifier1.save_pretrained("mod_tf")

**From above model evaluation,**

Model is able to predict even the sarcastic sentences and provides score of the sentiment which is great!!
Model is working perfectly fine and hence, can be used for deployment.

**Building Pipeline**

Making pipeline for the ML classification based models to automate the process of preprocessing, training & predicting the data.

In [ ]:
def clean_text_frame(X):
    b = clean_text
    return X.applymap(b)  # the function "clean_text" currently in your class.

In [ ]:
#Using the mapper to perform specific transformer to the features
m = DataFrameMapper([(['Review'], FunctionTransformer(func=clean_text_frame)),
                      
                    ], df_out=True, input_df=True)

In [ ]:
#Construct the column transfomer
vectorizer1 = TfidfVectorizer()
column_transformer = ColumnTransformer(
    [('Tfidf1', vectorizer1, 'Review')],
    remainder='passthrough')

In [ ]:
#Object Instantiation
mod = LogisticRegression(random_state=0, max_iter=1300)

mod1 = MultinomialNB()

sampling  = SMOTE(random_state=0)

In [ ]:
#Making Pipeline for Logistic Regression
model = Pipeline([ ('Cleaning', m),
                  ('Tfidf', column_transformer), ('SMOTE', sampling), ('MODEL', mod)
                ])

In [ ]:
#Reviewing the pipeline
model

In [ ]:
#Making Pipeline for Naive Bayes
model1 = Pipeline([ ('Cleaning', m),
                  ('Tfidf', column_transformer), ('SMOTE', sampling), ('MODEL', mod1)
                ])

In [ ]:
#Reviewing the pipeline
model1

**User Testing**


Testing the pipeline on our dataset

In [ ]:
#Making another copy of original data 
customer_review2 = customer_review.copy()

In [ ]:
#Label encoding the target variable
customer_review2['Rating'] = customer_review2['Rating'].apply(lambda x: 1 if x > 3 else 0)

In [ ]:
#Reviewing the data after label encoding
customer_review2.head()

In [ ]:
#Picking up 20 random samples from the original data for testing
usx = customer_review2.sample(n=20, random_state=12)
usx['Rating'].value_counts()

In [ ]:
#Index values of samples
usix = usx['Rating'].index

In [ ]:
#Splititing the collected samples for testing
X_t   = usx[['Review']]
y_t   = usx[['Rating']].values

In [ ]:
X_t.shape, y_t.shape

In [ ]:
#Dropping the collected samples from original dataset so that model is not trained on it
customer_review2 = customer_review2.drop(index=usix).reset_index()
customer_review2 = customer_review2.drop('index', axis=1)

In [ ]:
#Reviewing the data after dropping the test samples
customer_review2

In [ ]:
#Splitting the original data for training the model
X_tr = customer_review2[['Review']]
y_tr = customer_review2[['Rating']].values

In [ ]:
X_tr.head()

In [ ]:
y_tr

In [ ]:
#Training the model using Logistic Regression
model.fit(X_tr, y_tr)

In [ ]:
#Predicting the test sample
model.predict(X_t)

In [ ]:
#Model performance score
model.score(X_t, y_t)

In [ ]:
#Model Testing & Evaluation - Logistic Regression

print('Actual data:', '   ',y_t.flatten(), '\n')

print('Predicted data: ', model.predict(X_t))

In [ ]:
#model1 = model.fit_resample(X, y)

In [ ]:
#Training the model using Naive Bayes
model1.fit(X_tr, y_tr)

In [ ]:
#Predicting the test sample
model1.predict(X_t)

In [ ]:
#Model performance score
model1.score(X_t, y_t)

In [ ]:
#Model Testing & Evaluation - Naive Bayes

print('Actual data:', '   ',y_t.flatten(), '\n')

print('Predicted data: ', model1.predict(X_t))

# **Finalising the model**

# New Section

In [ ]:
from google.colab import files

with open('example.txt', 'w') as f:
  f.write('some content')

files.download('example.txt')

In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = 'REPLACE_WITH_YOUR_FILE_ID'
downloaded = drive.CreateFile({'id': file_id})
print('Downloaded content "{}"'.format(downloaded.GetContentString()))

In [ ]:
from google.colab import files

with open('example.txt', 'w') as f:
  f.write('some content')

files.download('example.txt')

In [ ]:
import IPython
from google.colab import output

display(IPython.display.Javascript('''
  window.someValue = new Promise(resolve => {
    setTimeout(() => {
      resolve("hello world!");
    }, 100);
  });
'''))


value = output.eval_js('someValue');
value

**Conclusion:**

After comparing the performance & accuracy for 2 classification models, both performed well. Since, the accuracy of Logistic Regression has higher precision & recall score we can finalise this model for our dataset.


We can also finalise the pre-trained model, as it was able to correctly judge the statement and also provide a sentiment score thereon.


Hence, we can go ahead and finalise our model 2, since dataset is not complex and overall stability and performance of model is good.

**Saving the model**

In [ ]:
#Custom reviews for testing after saving model
tt = {'Review': ['The bed was in such a good condition, I slept on the floor.', 'the food will bring nausea and motion sickness; it was that delicious sick.', 'It was a fucking amazing experience!! Will definitely visit this hotel again.']}
t1 = pd.DataFrame(tt, index=[0,1,2])
t1

In [ ]:
#Saving a joblib model
joblib.dump(model1, 'joblib_model1')

In [ ]:
#Loading a joblib model
mj = joblib.load('joblib_model1')

In [ ]:
#Predicting the model based on custom reviews
mj.predict(t1)

In [ ]:
#Saving the model to file
dump(model1, open('hrsa_intelligence.joblib', 'wb'))

In [ ]:
#Loading the file
ml = load(open('hrsa_intelligence.joblib', 'rb'))

In [ ]:
#Running the file for testing & evaluating custom reviews
print('Predicted Values:', ml.predict(t1))

In [ ]:
afrom google.colab import drive
drive.mount('/content/drive')